# Prepare DF

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

module_path

'/home/tjanicek/thesis/thoth-issue-predictor'

In [3]:
import graphviz
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz

from thoth_issue_predictor.utils.preprocessing import (
    prepare_df,
    create_python_version_packege_df,
)

FILE_NAME = "../datasets/thoth-performance-dataset-v2.0.zip"
inspections_df = prepare_df(FILE_NAME)
exit_codes = inspections_df["exit_code"]

INFO:thoth.report_processing.components.inspection:Retrieving dataset at path... /home/tjanicek/thesis/thoth-issue-predictor/thoth_issue_predictor/notebooks/inspections
INFO:thoth.report_processing.components.inspection:Considering inspection ID... inspection-tf-dm-np-605fcb35
INFO:thoth.report_processing.components.inspection:Considering inspection ID inspection-tf-dm-np-605fcb35.No files retrieved due to the following error: [Errno 2] No such file or directory: '/home/tjanicek/thesis/thoth-issue-predictor/thoth_issue_predictor/notebooks/inspections/inspection-tf-dm-np-605fcb35/results'
INFO:thoth.report_processing.components.inspection:Considering inspection ID... inspection-rhtf-py38-matmul-0c304a82-7txkr
INFO:thoth.report_processing.components.inspection:Considering inspection ID inspection-rhtf-py38-matmul-0c304a82-7txkr.No files retrieved due to the following error: [Errno 2] No such file or directory: '/home/tjanicek/thesis/thoth-issue-predictor/thoth_issue_predictor/notebooks/i

In [4]:
(
    issues_df,
    _,
    python_indexes,
) = create_python_version_packege_df(inspections_df=inspections_df)
issues_df["exit_code"] = exit_codes.astype("int")
issues_df = issues_df.loc[
    (issues_df.loc[:, issues_df.columns != "exit_code"] != 0).any(axis=1)
]

In [5]:
python_indexes

['unknown', 'pypi-org', 'pypi', 'aicoe']

In [6]:
print(f"Length of DF is {len(issues_df)}")
issues_df.head(5)

Length of DF is 110


,python,tensorflow_major,tensorflow_minor,tensorflow_patch,tensorflow_index,numpy_major,numpy_minor,numpy_patch,numpy_index,six_major,...,keras_index,pyyaml_major,pyyaml_minor,pyyaml_patch,pyyaml_index,tensorflow-cpu_major,tensorflow-cpu_minor,tensorflow-cpu_patch,tensorflow-cpu_index,exit_code
0,36,1,11,0,1,1,13,3,1,1,...,0,0,0,0,0,0,0,0,0,0
1,36,2,2,0,2,1,19,1,2,1,...,0,0,0,0,0,0,0,0,0,0
2,36,2,2,0,3,1,19,1,2,1,...,0,0,0,0,0,0,0,0,0,0
3,36,2,1,1,1,1,17,3,1,1,...,0,0,0,0,0,0,0,0,0,0
4,36,2,2,0,2,1,19,1,2,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
issues_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110 entries, 0 to 111
Columns: 174 entries, python to exit_code
dtypes: int64(174)
memory usage: 150.4 KB


# Failed inspections

In [8]:
failed_inspections = issues_df[issues_df["exit_code"] == 1]
print(f"Number of failed inspections: {len(failed_inspections)}")
failed_inspections.head()

Number of failed inspections: 4


,python,tensorflow_major,tensorflow_minor,tensorflow_patch,tensorflow_index,numpy_major,numpy_minor,numpy_patch,numpy_index,six_major,...,keras_index,pyyaml_major,pyyaml_minor,pyyaml_patch,pyyaml_index,tensorflow-cpu_major,tensorflow-cpu_minor,tensorflow-cpu_patch,tensorflow-cpu_index,exit_code
108,36,1,14,0,1,1,17,1,1,1,...,0,0,0,0,0,0,0,0,0,1
109,36,1,11,0,1,1,13,3,1,1,...,0,0,0,0,0,0,0,0,0,1
110,36,1,14,0,1,1,17,3,1,1,...,0,0,0,0,0,0,0,0,0,1
111,36,2,1,1,1,1,17,3,1,1,...,0,0,0,0,0,0,0,0,0,1


In [9]:
issue_x, issue_y = issues_df.drop("exit_code", axis=1), issues_df

# Decision Trees

In [10]:
clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, issue_x, issue_y["exit_code"], cv=10)

/home/tjanicek/.local/share/virtualenvs/thoth-issue-predictor-jSoEZjm6/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.81818182, 0.90909091, 0.90909091, 0.90909091])

In [11]:
clf.fit(issue_x, issue_y)

DecisionTreeClassifier(random_state=0)

In [14]:
dot_data = export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("ThothIssuePredictor")

'ThothIssuePredictor.pdf'